# Road Accident Risk Prediction

**Competition:** Kaggle Playground Series - Season 5, Episode 10

**Goal:** Predict accident risk (0-1) for different road types

**Evaluation Metric:** RMSE (Root Mean Squared Error)

**Target Score:** < 0.05 RMSE

## Strategy
1. Extensive Feature Engineering
2. Multiple Model Ensemble (CatBoost, XGBoost, LightGBM)
3. 5-Fold Cross-Validation
4. Weighted Averaging of Predictions

In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Libraries imported successfully!')

## 1. Load Data

In [ ]:
# Set paths
DATA_DIR = Path('../playground-series-s5e10')
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH = DATA_DIR / 'test.csv'
SUBMISSION_PATH = DATA_DIR / 'sample_submission.csv'

# Load data
print('Loading data...')
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
sample_submission = pd.read_csv(SUBMISSION_PATH)

print(f'Train shape: {train_df.shape}')
print(f'Test shape: {test_df.shape}')
print(f'\nFirst few rows:')
train_df.head()

In [ ]:
# Data overview
print('Column types:')
print(train_df.dtypes)
print(f'\nTarget statistics:')
print(train_df['accident_risk'].describe())
print(f'\nMissing values:')
print(train_df.isnull().sum().sum())

## 2. Feature Engineering

Creating advanced features to capture complex patterns:

In [ ]:
def create_features(df):
    """Create advanced features for the model."""
    df = df.copy()
    
    # Interaction features
    df['speed_per_lane'] = df['speed_limit'] / df['num_lanes']
    df['curvature_speed'] = df['curvature'] * df['speed_limit']
    df['accidents_per_lane'] = df['num_reported_accidents'] / df['num_lanes']
    
    # Boolean combinations
    df['risky_conditions'] = (
        (df['weather'] == 'rainy') & 
        (df['lighting'] == 'dim')
    ).astype(int)
    
    df['peak_holiday'] = (
        (df['holiday'] == True) & 
        (df['time_of_day'].isin(['evening', 'afternoon']))
    ).astype(int)
    
    df['school_morning'] = (
        (df['school_season'] == True) & 
        (df['time_of_day'] == 'morning')
    ).astype(int)
    
    # Risk score combinations
    df['high_speed_curve'] = (
        (df['speed_limit'] > 50) & 
        (df['curvature'] > 0.5)
    ).astype(int)
    
    df['no_signs_bad_weather'] = (
        (df['road_signs_present'] == False) & 
        (df['weather'] != 'clear')
    ).astype(int)
    
    # Polynomial features for continuous variables
    df['curvature_squared'] = df['curvature'] ** 2
    df['speed_squared'] = df['speed_limit'] ** 2
    df['accidents_squared'] = df['num_reported_accidents'] ** 2
    
    # Binned features
    df['speed_category'] = pd.cut(df['speed_limit'], 
                                    bins=[0, 35, 50, 70], 
                                    labels=['low', 'medium', 'high'])
    
    df['curvature_category'] = pd.cut(df['curvature'], 
                                       bins=[0, 0.33, 0.66, 1.0], 
                                       labels=['straight', 'moderate', 'sharp'])
    
    # Combined categorical features
    df['road_weather'] = df['road_type'] + '_' + df['weather']
    df['road_lighting'] = df['road_type'] + '_' + df['lighting']
    df['weather_time'] = df['weather'] + '_' + df['time_of_day']
    
    return df

print('Creating features...')
train_df = create_features(train_df)
test_df = create_features(test_df)
print('✓ Features created!')
print(f'New train shape: {train_df.shape}')

## 3. Prepare Data for Modeling

In [ ]:
# Define feature types
TARGET = 'accident_risk'
ID_COL = 'id'

# Categorical features
CATEGORICAL_COLS = [
    'road_type', 'lighting', 'weather', 'road_signs_present',
    'public_road', 'time_of_day', 'holiday', 'school_season',
    'speed_category', 'curvature_category',
    'road_weather', 'road_lighting', 'weather_time'
]

# Get all feature columns (excluding id and target)
FEATURE_COLS = [col for col in train_df.columns if col not in [ID_COL, TARGET]]

print(f'Total features: {len(FEATURE_COLS)}')
print(f'Categorical features: {len(CATEGORICAL_COLS)}')
print(f'Numerical features: {len(FEATURE_COLS) - len(CATEGORICAL_COLS)}')

In [ ]:
# Encode categorical features for XGBoost and LightGBM
def encode_categoricals(train, test, cat_cols):
    """Label encode categorical features."""
    train_encoded = train.copy()
    test_encoded = test.copy()
    
    encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        train_encoded[col] = le.fit_transform(train[col].astype(str))
        test_encoded[col] = le.transform(test[col].astype(str))
        encoders[col] = le
    
    return train_encoded, test_encoded, encoders

# Prepare datasets
X = train_df[FEATURE_COLS].copy()
y = train_df[TARGET].copy()
X_test = test_df[FEATURE_COLS].copy()

# Encoded version for XGBoost/LightGBM
X_encoded, X_test_encoded, encoders = encode_categoricals(X, X_test, CATEGORICAL_COLS)

print('✓ Data prepared for modeling!')

## 4. Model Training with Cross-Validation

Training multiple models with 5-fold CV:

In [ ]:
# Cross-validation setup
N_FOLDS = 5
RANDOM_STATE = 42
kfold = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# Storage for predictions
oof_catboost = np.zeros(len(X))
oof_xgboost = np.zeros(len(X))
oof_lightgbm = np.zeros(len(X))

test_catboost = np.zeros(len(X_test))
test_xgboost = np.zeros(len(X_test))
test_lightgbm = np.zeros(len(X_test))

fold_scores = {'catboost': [], 'xgboost': [], 'lightgbm': []}

print('Starting cross-validation training...\n')

In [ ]:
# Train models with progress tracking
for fold, (train_idx, valid_idx) in enumerate(tqdm(list(kfold.split(X)), desc='Folds'), 1):
    print(f'\n{"="*60}')
    print(f'Fold {fold}/{N_FOLDS}')
    print(f'{"="*60}')
    
    # Split data
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    X_train_enc, X_valid_enc = X_encoded.iloc[train_idx], X_encoded.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # ===== CatBoost =====
    print('\n[1/3] Training CatBoost...')
    cat_features = [X_train.columns.get_loc(col) for col in CATEGORICAL_COLS]
    
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    valid_pool = Pool(X_valid, y_valid, cat_features=cat_features)
    test_pool = Pool(X_test, cat_features=cat_features)
    
    cb_model = CatBoostRegressor(
        iterations=3000,
        learning_rate=0.03,
        depth=8,
        loss_function='RMSE',
        eval_metric='RMSE',
        subsample=0.8,
        colsample_bylevel=0.8,
        random_seed=RANDOM_STATE + fold,
        verbose=False
    )
    
    cb_model.fit(
        train_pool,
        eval_set=valid_pool,
        early_stopping_rounds=200,
        verbose=False
    )
    
    oof_catboost[valid_idx] = cb_model.predict(valid_pool)
    test_catboost += cb_model.predict(test_pool) / N_FOLDS
    
    cb_score = mean_squared_error(y_valid, oof_catboost[valid_idx], squared=False)
    fold_scores['catboost'].append(cb_score)
    print(f'  CatBoost RMSE: {cb_score:.5f}')
    
    # ===== XGBoost =====
    print('\n[2/3] Training XGBoost...')
    xgb_model = XGBRegressor(
        n_estimators=3000,
        learning_rate=0.03,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=RANDOM_STATE + fold,
        tree_method='hist',
        enable_categorical=True
    )
    
    xgb_model.fit(
        X_train_enc, y_train,
        eval_set=[(X_valid_enc, y_valid)],
        verbose=False
    )
    
    oof_xgboost[valid_idx] = xgb_model.predict(X_valid_enc)
    test_xgboost += xgb_model.predict(X_test_encoded) / N_FOLDS
    
    xgb_score = mean_squared_error(y_valid, oof_xgboost[valid_idx], squared=False)
    fold_scores['xgboost'].append(xgb_score)
    print(f'  XGBoost RMSE: {xgb_score:.5f}')
    
    # ===== LightGBM =====
    print('\n[3/3] Training LightGBM...')
    lgb_model = LGBMRegressor(
        n_estimators=3000,
        learning_rate=0.03,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        random_state=RANDOM_STATE + fold,
        verbose=-1
    )
    
    lgb_model.fit(
        X_train_enc, y_train,
        eval_set=[(X_valid_enc, y_valid)],
        callbacks=[]
    )
    
    oof_lightgbm[valid_idx] = lgb_model.predict(X_valid_enc)
    test_lightgbm += lgb_model.predict(X_test_encoded) / N_FOLDS
    
    lgb_score = mean_squared_error(y_valid, oof_lightgbm[valid_idx], squared=False)
    fold_scores['lightgbm'].append(lgb_score)
    print(f'  LightGBM RMSE: {lgb_score:.5f}')

print(f'\n{"="*60}')
print('Training complete!')
print(f'{"="*60}')

## 5. Cross-Validation Results

In [ ]:
# Calculate CV scores
cv_catboost = mean_squared_error(y, oof_catboost, squared=False)
cv_xgboost = mean_squared_error(y, oof_xgboost, squared=False)
cv_lightgbm = mean_squared_error(y, oof_lightgbm, squared=False)

print('\n' + '='*60)
print('CROSS-VALIDATION RESULTS')
print('='*60)
print(f'CatBoost  CV RMSE: {cv_catboost:.5f}')
print(f'XGBoost   CV RMSE: {cv_xgboost:.5f}')
print(f'LightGBM  CV RMSE: {cv_lightgbm:.5f}')
print('='*60)

# Fold-wise scores
print('\nFold-wise scores:')
fold_df = pd.DataFrame(fold_scores)
fold_df.index = [f'Fold {i+1}' for i in range(N_FOLDS)]
fold_df.loc['Mean'] = fold_df.mean()
fold_df.loc['Std'] = fold_df.std()
print(fold_df.round(5))

In [ ]:
# Plot CV scores
fig, ax = plt.subplots(figsize=(10, 6))
fold_df.iloc[:-2].plot(kind='bar', ax=ax, width=0.8)
ax.set_title('Cross-Validation RMSE by Fold', fontsize=14, fontweight='bold')
ax.set_xlabel('Fold', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.legend(title='Model', fontsize=10)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 6. Ensemble Predictions

Combining predictions from all three models:

In [ ]:
# Try different ensemble weights
print('Testing ensemble weights...\n')

best_score = float('inf')
best_weights = None

# Test different weight combinations
weight_options = [
    (0.4, 0.3, 0.3),  # Equal-ish
    (0.5, 0.25, 0.25),  # Favor CatBoost
    (0.34, 0.33, 0.33),  # Exactly equal
    (0.6, 0.2, 0.2),  # Heavy CatBoost
    (0.45, 0.35, 0.2),  # Favor top 2
]

for weights in weight_options:
    w_cb, w_xgb, w_lgb = weights
    
    oof_ensemble = (
        w_cb * oof_catboost +
        w_xgb * oof_xgboost +
        w_lgb * oof_lightgbm
    )
    
    ensemble_score = mean_squared_error(y, oof_ensemble, squared=False)
    
    print(f'Weights {weights}: RMSE = {ensemble_score:.5f}')
    
    if ensemble_score < best_score:
        best_score = ensemble_score
        best_weights = weights

print(f'\n✓ Best weights: {best_weights}')
print(f'✓ Best ensemble CV RMSE: {best_score:.5f}')

In [ ]:
# Create final ensemble predictions
w_cb, w_xgb, w_lgb = best_weights

test_ensemble = (
    w_cb * test_catboost +
    w_xgb * test_xgboost +
    w_lgb * test_lightgbm
)

# Clip predictions to valid range [0, 1]
test_ensemble = np.clip(test_ensemble, 0, 1)

print(f'Final test predictions:')
print(f'  Min: {test_ensemble.min():.4f}')
print(f'  Max: {test_ensemble.max():.4f}')
print(f'  Mean: {test_ensemble.mean():.4f}')
print(f'  Median: {np.median(test_ensemble):.4f}')

## 7. Create Submission File

In [ ]:
# Create submission
submission = sample_submission.copy()
submission['accident_risk'] = test_ensemble

# Save to file
output_path = Path('../submission.csv')
submission.to_csv(output_path, index=False)

print('✓ Submission file created successfully!')
print(f'  Location: {output_path.resolve()}')
print(f'  Shape: {submission.shape}')
print(f'\nFirst few rows:')
print(submission.head(10))

## 8. Analysis & Visualizations

In [ ]:
# Plot prediction distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Training target distribution
axes[0, 0].hist(y, bins=50, alpha=0.7, color='blue', edgecolor='black')
axes[0, 0].set_title('Training Target Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Accident Risk')
axes[0, 0].set_ylabel('Frequency')

# Test predictions distribution
axes[0, 1].hist(test_ensemble, bins=50, alpha=0.7, color='green', edgecolor='black')
axes[0, 1].set_title('Test Predictions Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Predicted Accident Risk')
axes[0, 1].set_ylabel('Frequency')

# OOF vs Actual scatter
oof_ensemble = (
    w_cb * oof_catboost +
    w_xgb * oof_xgboost +
    w_lgb * oof_lightgbm
)
axes[1, 0].scatter(y, oof_ensemble, alpha=0.3, s=1)
axes[1, 0].plot([0, 1], [0, 1], 'r--', lw=2)
axes[1, 0].set_title('OOF Predictions vs Actual', fontweight='bold')
axes[1, 0].set_xlabel('Actual Accident Risk')
axes[1, 0].set_ylabel('Predicted Accident Risk')

# Model comparison
model_scores = pd.Series({
    'CatBoost': cv_catboost,
    'XGBoost': cv_xgboost,
    'LightGBM': cv_lightgbm,
    'Ensemble': best_score
})
model_scores.plot(kind='bar', ax=axes[1, 1], color=['skyblue', 'lightgreen', 'lightcoral', 'gold'])
axes[1, 1].set_title('Model Performance Comparison', fontweight='bold')
axes[1, 1].set_ylabel('CV RMSE')
axes[1, 1].set_xlabel('Model')
axes[1, 1].grid(axis='y', alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
residuals = y - oof_ensemble

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residual distribution
axes[0].hist(residuals, bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[0].set_title('Residual Distribution', fontweight='bold')
axes[0].set_xlabel('Residual (Actual - Predicted)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(0, color='red', linestyle='--', linewidth=2)

# Residual scatter
axes[1].scatter(oof_ensemble, residuals, alpha=0.3, s=1)
axes[1].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_title('Residual Plot', fontweight='bold')
axes[1].set_xlabel('Predicted Accident Risk')
axes[1].set_ylabel('Residual')

plt.tight_layout()
plt.show()

print(f'Residual statistics:')
print(f'  Mean: {residuals.mean():.6f}')
print(f'  Std: {residuals.std():.6f}')
print(f'  Min: {residuals.min():.6f}')
print(f'  Max: {residuals.max():.6f}')

## Summary

**Final Results:**
- Ensemble model combining CatBoost, XGBoost, and LightGBM
- 5-fold cross-validation
- Extensive feature engineering (27 features)
- Submission file created: `submission.csv`

**Next Steps:**
1. Upload submission to Kaggle
2. Monitor leaderboard score
3. If score > 0.05, iterate with:
   - More feature engineering
   - Hyperparameter tuning
   - Include original dataset
   - Advanced stacking techniques